In [ ]:
#imports
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

ExERCÍCIO 1 – CLASSIFICAÇÃO MULTICLASSE
Dataset: Wine Dataset (UCI)
1. Treinar uma rede neural em Keras para classificar vinhos em 3 classes.
- Configuração mínima: 2 camadas ocultas com 32 neurônios cada, função de ativação ReLU.
- Camada de saída com 3 neurônios, função de ativação Softmax.
- Função de perda: categorical_crossentropy.
- Otimizador: Adam.
2. Comparar os resultados com um modelo do scikit-learn (RandomForestClassifier ou
LogisticRegression).
3. Registrar métricas de acurácia e discutir qual modelo teve melhor desempenho

EXERCÍCIO 2 – REGRESSÃO
Dataset: California Housing Dataset (Scikit-learn)
1. Treinar uma rede neural em Keras para prever o valor médio das casas.
- Configuração mínima: 3 camadas ocultas com 64, 32 e 16 neurônios, função de ativação ReLU.
- Camada de saída com 1 neurônio, função de ativação Linear.
- Função de perda: mse.
- Otimizador: Adam.
2. Comparar os resultados com um modelo do scikit-learn (LinearRegression ou
RandomForestRegressor).
3. Registrar métricas de erro (RMSE ou MAE) e discutir qual modelo teve melhor desempenho

In [ ]:
# 1. Carregar e preparar os dados
print("Carregando o dataset California Housing...")
housing = fetch_california_housing()
X, y = housing.data, housing.target

Carregando o dataset California Housing...


In [ ]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
# Escalar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Shape dos dados de treino: {X_train.shape}")
print(f"Shape dos dados de teste: {X_test.shape}")
print(f"Nomes das features: {housing.feature_names}")

Shape dos dados de treino: (16512, 8)
Shape dos dados de teste: (4128, 8)
Nomes das features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [ ]:
# 2. Treinar modelo Keras
print("\n" + "="*50)
print("Treinando Rede Neural Keras...")
print("="*50)

def create_keras_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        layers.Dense(32, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='linear')
    ])

    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    return model

# Criar e treinar o modelo
keras_model = create_keras_model()

print("Arquitetura da rede neural:")
keras_model.summary()

# Callback para early stopping
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True)


Treinando Rede Neural Keras...
Arquitetura da rede neural:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,201 (12.50 KB)

 Trainable params: 3,201 (12.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Treinar o modelo
history = keras_model.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1)

# Fazer previsões
y_pred_keras = keras_model.predict(X_test_scaled).flatten()

# 3. Treinar modelos scikit-learn
print("\n" + "="*50)
print("Treinando modelos scikit-learn...")
print("="*50)

# Linear Regression
print("Treinando Linear Regression...")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)

# Random Forest Regressor
print("Treinando Random Forest Regressor...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)  # Random Forest não precisa de scaling
y_pred_rf = rf_model.predict(X_test)

Epoch 1/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.3873 - mae: 0.8109 - val_loss: 0.4480 - val_mae: 0.4931
Epoch 2/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4017 - mae: 0.4516 - val_loss: 0.3927 - val_mae: 0.4488
Epoch 3/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3475 - mae: 0.4208 - val_loss: 0.3684 - val_mae: 0.4289
Epoch 4/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.3372 - mae: 0.4142 - val_loss: 0.3569 - val_mae: 0.4127
Epoch 5/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3440 - mae: 0.4170 - val_loss: 0.3580 - val_mae: 0.4308
Epoch 6/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3196 - mae: 0.3996 - val_loss: 0.3398 - val_mae: 0.4020
Epoch 7/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3056 - mae: 0.3868 - val_loss: 0.3398 - val_mae: 0.4120
Epoch 8/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.2984 - mae: 0.3805 - val_loss: 0.3253 - val_mae: 0.3991
Epoch 9/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/

In [ ]:
# 4. Avaliar e comparar modelos
print("\n" + "="*50)
print("COMPARAÇÃO DOS MODELOS")
print("="*50)

def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)

    print(f"\n{model_name}:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  MSE:  {mse:.4f}")

    return rmse, mae

# Avaliar todos os modelos
results = {}
results['Keras NN'] = evaluate_model(y_test, y_pred_keras, "Keras Neural Network")
results['Linear Regression'] = evaluate_model(y_test, y_pred_lr, "Linear Regression")
results['Random Forest'] = evaluate_model(y_test, y_pred_rf, "Random Forest")

# 5. Visualizar resultados
plt.figure(figsize=(15, 5))

# Gráfico 1: Comparação de previsões vs valores reais
plt.subplot(1, 3, 1)
plt.scatter(y_test, y_pred_keras, alpha=0.5, label='Keras NN')
plt.scatter(y_test, y_pred_rf, alpha=0.5, label='Random Forest')
plt.scatter(y_test, y_pred_lr, alpha=0.5, label='Linear Regression')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Valor Real')
plt.ylabel('Valor Previsto')
plt.title('Previsões vs Valores Reais')
plt.legend()

# Gráfico 2: Comparação de RMSE
plt.subplot(1, 3, 2)
models = list(results.keys())
rmse_values = [results[model][0] for model in models]
plt.bar(models, rmse_values, color=['blue', 'orange', 'green'])
plt.title('Comparação do RMSE')
plt.xticks(rotation=45)
plt.ylabel('RMSE')

# Gráfico 3: Comparação de MAE
plt.subplot(1, 3, 3)
mae_values = [results[model][1] for model in models]
plt.bar(models, mae_values, color=['blue', 'orange', 'green'])
plt.title('Comparação do MAE')
plt.xticks(rotation=45)
plt.ylabel('MAE')

plt.tight_layout()
plt.show()

# 6. Análise detalhada
print("\n" + "="*50)
print("ANÁLISE DOS RESULTADOS")
print("="*50)

# Encontrar o melhor modelo baseado no RMSE
best_model_rmse = min(results, key=lambda x: results[x][0])
best_model_mae = min(results, key=lambda x: results[x][1])

print(f"\nMelhor modelo por RMSE: {best_model_rmse} (RMSE = {results[best_model_rmse][0]:.4f})")
print(f"Melhor modelo por MAE: {best_model_mae} (MAE = {results[best_model_mae][1]:.4f})")

# Calcular porcentagem de melhoria
rmse_baseline = results['Linear Regression'][0]
for model_name, (rmse, mae) in results.items():
    if model_name != 'Linear Regression':
        improvement = ((rmse_baseline - rmse) / rmse_baseline) * 100
        print(f"\n{model_name} vs Linear Regression:")
        print(f"  Melhoria no RMSE: {improvement:+.2f}%")

# 7. Visualizar histórico de treinamento da rede neural
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Evolução da Loss durante o Treinamento')
plt.xlabel('Época')
plt.ylabel('Loss (MSE)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Evolução do MAE durante o Treinamento')
plt.xlabel('Época')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

# 8. Importância das features (Random Forest)
print("\n" + "="*50)
print("IMPORTÂNCIA DAS FEATURES (Random Forest)")
print("="*50)

feature_importance = pd.DataFrame({
    'feature': housing.feature_names,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance)

In [ ]:
#Conclusão:
#O modelo Random Forest costuma ter uma performance melhor quando se trata de problemas de regressão com dados tabulares, mas a rede neural pode alcançar resultados semelhantes com a arquitetura e tuning adequados. A Regressão Linear serve como uma baseline útil para acompanhamento de possíveis melhorias.

Comparação de desempenho entre os modelos